In [1]:
import pandas as pd

# Load the CSV file
file_path = 'SPY ETF Stock Price History.csv'
spy_data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
spy_data.head()


,Date,Price,Open,High,Low,Vol.,Change %
0,08/30/2024,563.68,560.77,564.20,557.14,62.70M,0.95%
1,08/29/2024,558.35,560.31,563.68,557.18,38.72M,0.01%
2,08/28/2024,558.30,561.21,561.65,555.04,41.07M,-0.58%
3,08/27/2024,561.56,559.49,562.06,558.32,32.69M,0.14%
4,08/26/2024,560.79,563.18,563.91,559.05,35.79M,-0.24%


In [2]:
# Clean the data

# Convert the 'Date' column to datetime format
spy_data['Date'] = pd.to_datetime(spy_data['Date'], format='%m/%d/%Y')

# Remove commas from the 'Vol.' column and convert it to numeric
spy_data['Vol.'] = spy_data['Vol.'].str.replace('M', '').str.replace('B', '').str.replace(',', '')
spy_data['Vol.'] = pd.to_numeric(spy_data['Vol.'], errors='coerce')

# Convert 'Vol.' to appropriate scale (M for millions, B for billions)
# Assuming the original volume data is in millions
spy_data['Vol.'] = spy_data['Vol.'] * 1e6

# Remove percentage sign from 'Change %' column and convert it to numeric
spy_data['Change %'] = spy_data['Change %'].str.replace('%', '')
spy_data['Change %'] = pd.to_numeric(spy_data['Change %'], errors='coerce')

# Display the cleaned data
spy_data.head()


,Date,Price,Open,High,Low,Vol.,Change %
0,2024-08-30,563.68,560.77,564.20,557.14,62700000.0,0.95
1,2024-08-29,558.35,560.31,563.68,557.18,38720000.0,0.01
2,2024-08-28,558.30,561.21,561.65,555.04,41070000.0,-0.58
3,2024-08-27,561.56,559.49,562.06,558.32,32690000.0,0.14
4,2024-08-26,560.79,563.18,563.91,559.05,35790000.0,-0.24


In [3]:
# Filter the data for days with more than 1% movement
movement_days = spy_data[spy_data['Change %'].abs() > 1]

# Display the filtered data
movement_days.head()


,Date,Price,Open,High,Low,Vol.,Change %
5,2024-08-23,562.13,559.53,563.09,557.29,50640000.0,1.06
11,2024-08-15,553.07,549.50,553.36,548.88,60850000.0,1.71
13,2024-08-13,542.04,536.53,542.28,536.28,52330000.0,1.64
16,2024-08-08,530.65,523.91,531.29,521.84,63280000.0,2.31
19,2024-08-05,517.38,511.64,523.58,510.27,146270000.0,-2.91


In [5]:
# Re-initialize lists to store trade details
trade_type = []
entry_price = []
stop_loss = []
take_profit = []

# Loop over each row in the filtered movement_days
for i in range(1, len(movement_days)):
    row = movement_days.iloc[i]
    prev_row = spy_data[spy_data['Date'] == row['Date'] - pd.Timedelta(days=1)]
    
    if not prev_row.empty:
        # Check if it's a buy or sell condition
        if row['Change %'] > 1:
            trade_type.append('Buy')
            entry_price.append(row['Price'])
            sl = prev_row['Low'].values[0]  # Stop loss below yesterday's low
            stop_loss.append(sl)
            tp = row['Price'] + 2 * (row['Price'] - sl)  # Take profit at 1:2 risk/reward
            take_profit.append(tp)
        elif row['Change %'] < -1:
            trade_type.append('Sell')
            entry_price.append(row['Price'])
            sl = prev_row['High'].values[0]  # Stop loss above yesterday's high
            stop_loss.append(sl)
            tp = row['Price'] - 2 * (sl - row['Price'])  # Take profit at 1:2 risk/reward
            take_profit.append(tp)
    else:
        trade_type.append(None)
        entry_price.append(None)
        stop_loss.append(None)
        take_profit.append(None)

# Ensure that the lists align with the dataframe's length
if len(trade_type) < len(movement_days):
    trade_type.insert(0, None)
    entry_price.insert(0, None)
    stop_loss.insert(0, None)
    take_profit.insert(0, None)

# Add these details to the DataFrame
movement_days['Trade Type'] = trade_type
movement_days['Entry Price'] = entry_price
movement_days['Stop Loss'] = stop_loss
movement_days['Take Profit'] = take_profit

# Display the updated DataFrame with trade details
movement_days.head()


C:\Users\16469\AppData\Local\Temp\ipykernel_28724\1213555691.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movement_days['Trade Type'] = trade_type
C:\Users\16469\AppData\Local\Temp\ipykernel_28724\1213555691.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movement_days['Entry Price'] = entry_price
C:\Users\16469\AppData\Local\Temp\ipykernel_28724\1213555691.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,Date,Price,Open,High,Low,Vol.,Change %,Trade Type,Entry Price,Stop Loss,Take Profit
5,2024-08-23,562.13,559.53,563.09,557.29,50640000.0,1.06,None,NaN,NaN,NaN
11,2024-08-15,553.07,549.50,553.36,548.88,60850000.0,1.71,Buy,553.07,540.12,578.97
13,2024-08-13,542.04,536.53,542.28,536.28,52330000.0,1.64,Buy,542.04,530.95,564.22
16,2024-08-08,530.65,523.91,531.29,521.84,63280000.0,2.31,Buy,530.65,518.05,555.85
19,2024-08-05,517.38,511.64,523.58,510.27,146270000.0,-2.91,None,NaN,NaN,NaN


In [6]:
# Initialize the starting capital
starting_capital = 100000
capital = starting_capital
position_size = 10000  # Position size per trade (10% of capital)
profits = []

# Simulate each trade
for i in range(len(movement_days)):
    trade = movement_days.iloc[i]
    
    if trade['Trade Type'] == 'Buy':
        # Simulate buy trade
        if trade['Low'] <= trade['Stop Loss']:
            # Stop loss hit
            profit = position_size * ((trade['Stop Loss'] - trade['Entry Price']) / trade['Entry Price'])
        elif trade['High'] >= trade['Take Profit']:
            # Take profit hit
            profit = position_size * ((trade['Take Profit'] - trade['Entry Price']) / trade['Entry Price'])
        else:
            # Neither hit, assume no profit
            profit = 0
    elif trade['Trade Type'] == 'Sell':
        # Simulate sell trade
        if trade['High'] >= trade['Stop Loss']:
            # Stop loss hit
            profit = position_size * ((trade['Entry Price'] - trade['Stop Loss']) / trade['Entry Price'])
        elif trade['Low'] <= trade['Take Profit']:
            # Take profit hit
            profit = position_size * ((trade['Entry Price'] - trade['Take Profit']) / trade['Entry Price'])
        else:
            # Neither hit, assume no profit
            profit = 0
    else:
        profit = 0
    
    profits.append(profit)
    capital += profit

# Add the profits to the DataFrame
movement_days['Profit'] = profits

# Display the final account balance and some of the trades
final_balance = capital
movement_days[['Date', 'Trade Type', 'Entry Price', 'Stop Loss', 'Take Profit', 'Profit']].head(), final_balance


C:\Users\16469\AppData\Local\Temp\ipykernel_28724\1531316085.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movement_days['Profit'] = profits


(         Date Trade Type  Entry Price  Stop Loss  Take Profit  Profit
 5  2024-08-23       None          NaN        NaN          NaN     0.0
 11 2024-08-15        Buy       553.07     540.12       578.97     0.0
 13 2024-08-13        Buy       542.04     530.95       564.22     0.0
 16 2024-08-08        Buy       530.65     518.05       555.85     0.0
 19 2024-08-05       None          NaN        NaN          NaN     0.0,
 82766.68353893746)

In [7]:
# Initialize the starting capital
starting_capital = 100000
capital = starting_capital
position_size = 10000  # Position size per trade (10% of capital)
profits = []

# Simulate each trade with a 5-day time allowance
for i in range(len(movement_days)):
    trade = movement_days.iloc[i]
    
    if trade['Trade Type'] in ['Buy', 'Sell']:
        trade_end_date = trade['Date'] + pd.Timedelta(days=5)
        trade_period = spy_data[(spy_data['Date'] > trade['Date']) & (spy_data['Date'] <= trade_end_date)]
        
        if not trade_period.empty:
            if trade['Trade Type'] == 'Buy':
                # Check if stop loss or take profit is hit within 5 days
                if any(trade_period['Low'] <= trade['Stop Loss']):
                    # Stop loss hit
                    sl_price = trade['Stop Loss']
                    profit = position_size * ((sl_price - trade['Entry Price']) / trade['Entry Price'])
                elif any(trade_period['High'] >= trade['Take Profit']):
                    # Take profit hit
                    tp_price = trade['Take Profit']
                    profit = position_size * ((tp_price - trade['Entry Price']) / trade['Entry Price'])
                else:
                    # Neither hit, exit at the closing price on the 5th day
                    close_price = trade_period.iloc[-1]['Price']
                    profit = position_size * ((close_price - trade['Entry Price']) / trade['Entry Price'])
            
            elif trade['Trade Type'] == 'Sell':
                # Check if stop loss or take profit is hit within 5 days
                if any(trade_period['High'] >= trade['Stop Loss']):
                    # Stop loss hit
                    sl_price = trade['Stop Loss']
                    profit = position_size * ((trade['Entry Price'] - sl_price) / trade['Entry Price'])
                elif any(trade_period['Low'] <= trade['Take Profit']):
                    # Take profit hit
                    tp_price = trade['Take Profit']
                    profit = position_size * ((trade['Entry Price'] - tp_price) / trade['Entry Price'])
                else:
                    # Neither hit, exit at the closing price on the 5th day
                    close_price = trade_period.iloc[-1]['Price']
                    profit = position_size * ((trade['Entry Price'] - close_price) / trade['Entry Price'])
        else:
            profit = 0
        
        profits.append(profit)
        capital += profit
    else:
        profits.append(0)

# Add the profits to the DataFrame
movement_days['Profit'] = profits

# Display the final account balance and some of the trades
final_balance = capital
movement_days[['Date', 'Trade Type', 'Entry Price', 'Stop Loss', 'Take Profit', 'Profit']].head(), final_balance


C:\Users\16469\AppData\Local\Temp\ipykernel_28724\2166036099.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movement_days['Profit'] = profits


(         Date Trade Type  Entry Price  Stop Loss  Take Profit     Profit
 5  2024-08-23       None          NaN        NaN          NaN   0.000000
 11 2024-08-15        Buy       553.07     540.12       578.97  22.420308
 13 2024-08-13        Buy       542.04     530.95       564.22  31.547487
 16 2024-08-08        Buy       530.65     518.05       555.85  44.096862
 19 2024-08-05       None          NaN        NaN          NaN   0.000000,
 84276.36646420334)

In [8]:
# Initialize the starting capital and variables for trailing stop loss
starting_capital = 100000
capital = starting_capital
position_size = 10000  # Position size per trade (10% of capital)
profits = []

# Function to update trailing stop loss
def update_trailing_stop(entry_price, current_price, sl_price, trade_type):
    if trade_type == 'Buy':
        move = current_price - entry_price
        if move >= 0.01 * entry_price:
            sl_price = max(sl_price, entry_price + 0.5 * move)
    elif trade_type == 'Sell':
        move = entry_price - current_price
        if move >= 0.01 * entry_price:
            sl_price = min(sl_price, entry_price - 0.5 * move)
    return sl_price

# Simulate each trade with trailing stop loss and 5-day time allowance
for i in range(len(movement_days)):
    trade = movement_days.iloc[i]
    
    if trade['Trade Type'] in ['Buy', 'Sell']:
        trade_end_date = trade['Date'] + pd.Timedelta(days=5)
        trade_period = spy_data[(spy_data['Date'] > trade['Date']) & (spy_data['Date'] <= trade_end_date)]
        sl_price = trade['Stop Loss']
        profit = 0
        
        if not trade_period.empty:
            for j in range(len(trade_period)):
                day = trade_period.iloc[j]
                
                # Update trailing stop loss
                sl_price = update_trailing_stop(trade['Entry Price'], day['Price'], sl_price, trade['Trade Type'])
                
                if trade['Trade Type'] == 'Buy':
                    if day['Low'] <= sl_price:
                        # Stop loss hit
                        profit = position_size * ((sl_price - trade['Entry Price']) / trade['Entry Price'])
                        break
                    elif day['High'] >= trade['Take Profit']:
                        # Take profit hit
                        profit = position_size * ((trade['Take Profit'] - trade['Entry Price']) / trade['Entry Price'])
                        break
                elif trade['Trade Type'] == 'Sell':
                    if day['High'] >= sl_price:
                        # Stop loss hit
                        profit = position_size * ((trade['Entry Price'] - sl_price) / trade['Entry Price'])
                        break
                    elif day['Low'] <= trade['Take Profit']:
                        # Take profit hit
                        profit = position_size * ((trade['Entry Price'] - trade['Take Profit']) / trade['Entry Price'])
                        break
            else:
                # If no stop loss or take profit hit, exit at the closing price on the 5th day
                close_price = trade_period.iloc[-1]['Price']
                if trade['Trade Type'] == 'Buy':
                    profit = position_size * ((close_price - trade['Entry Price']) / trade['Entry Price'])
                elif trade['Trade Type'] == 'Sell':
                    profit = position_size * ((trade['Entry Price'] - close_price) / trade['Entry Price'])
        
        profits.append(profit)
        capital += profit
    else:
        profits.append(0)

# Add the profits to the DataFrame
movement_days['Profit'] = profits

# Display the final account balance and some of the trades
final_balance = capital
movement_days[['Date', 'Trade Type', 'Entry Price', 'Stop Loss', 'Take Profit', 'Profit']].head(), final_balance


C:\Users\16469\AppData\Local\Temp\ipykernel_28724\3800606210.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movement_days['Profit'] = profits


(         Date Trade Type  Entry Price  Stop Loss  Take Profit      Profit
 5  2024-08-23       None          NaN        NaN          NaN    0.000000
 11 2024-08-15        Buy       553.07     540.12       578.97   59.124523
 13 2024-08-13        Buy       542.04     530.95       564.22  113.183529
 16 2024-08-08        Buy       530.65     518.05       555.85  107.321210
 19 2024-08-05       None          NaN        NaN          NaN    0.000000,
 93663.75865529638)